# IEOR 142 - Final Project

In [258]:
#Loading Dependencies
!pip install statsmodels
import pandas as pd
import numpy as np
import datetime
import statsmodels.api as sm
import statsmodels.formula.api as smf
import random

from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix

In [259]:
"""Functions"""

def time_since(i):
    c = i.split(' -')[0]
    format = '%b %d %Y'
    a = datetime.datetime.strptime(c, format)
    b = datetime.datetime.today() - a
    return b.days

def winner(home_score, away_score):
    if home_score>away_score:
        return 1
    if home_score<away_score:
        return 0
    if home_score==away_score:
        return 0.5

def winner_binary(home_score, away_score):
    if home_score>away_score:
        return 1
    return 0


def year(text):
    return text.split('-')[0]

In [260]:
#Kaggle Data
fifa_ranking = pd.read_csv('fifa_ranking-2022-10-06.csv') 
goalscorers = pd.read_csv('goalscorers.csv') 
international_matches = pd.read_csv('international_matches.csv')
last_team_scores = pd.read_csv('last_team_scores.csv')
players_22 = pd.read_csv('players_22.csv')
qatar_group_stage = pd.read_csv('Qatar_group_stage.csv')
results = pd.read_csv('results.csv')
shootouts = pd.read_csv('shootouts.csv')
squad_stats = pd.read_csv('squad_stats.csv')

/var/folders/7r/3wbh_3yx46vflflwjp0xxbkm0000gn/T/ipykernel_83543/1720374977.py:6: DtypeWarning: Columns (25,108) have mixed types. Specify dtype option on import or set low_memory=False.
  players_22 = pd.read_csv('players_22.csv')


In [261]:
fifa_ranking['year'] = fifa_ranking['rank_date'].apply(year)
fifa_ranking['year'] = fifa_ranking['year'].apply(int)
fifa_ranking[fifa_ranking['year']>=2014].groupby(by='country_full').mean()

,rank,total_points,previous_points,rank_change,year
country_full,,,,,
Afghanistan,146.280899,530.412360,530.872584,0.179775,2017.41573
Albania,57.191011,908.973034,909.679663,0.101124,2017.41573
Algeria,38.831461,1037.677640,1039.811348,0.112360,2017.41573
American Samoa,189.876404,387.497079,387.359213,-0.089888,2017.41573
Andorra,169.146067,457.956854,456.527640,-0.528090,2017.41573
...,...,...,...,...,...
Vietnam,118.157303,628.651461,626.700337,-0.550562,2017.41573
Wales,21.707865,1179.028764,1174.390337,-0.426966,2017.41573
Yemen,153.382022,509.252360,508.302697,-0.269663,2017.41573


In [262]:
#Match to Match Data
#Kian
asia_cup = pd.read_csv('GitHub Data/international-asian-cup-qualification-matches-2023-to-2023-stats.csv')
gold_cup = pd.read_csv('GitHub Data/international-concacaf-gold-cup-matches-2021-to-2021-stats.csv')
copa = pd.read_csv('GitHub Data/international-copa-america-matches-2021-to-2021-stats.csv')
arab_cup = pd.read_csv('GitHub Data/international-fifa-arab-cup-matches-2021-to-2021-stats.csv')
#conf_cup = pd.read_csv('GitHub Data/international-fifa-confederations-cup-matches-2017-to-2017-stats.csv')
#wc_2002 = pd.read_csv('/work/GitHub Data/international-fifa-world-cup-2002-korea-japan-matches-2002-to-2002-stats.csv')
#wc_2006 = pd.read_csv('/work/GitHub Data/international-fifa-world-cup-2006-germany-matches-2006-to-2006-stats.csv')
#wc_2010 = pd.read_csv('/work/GitHub Data/international-fifa-world-cup-2010-south-africa-matches-2010-to-2010-stats.csv')
wc_2014 = pd.read_csv('GitHub Data/international-fifa-world-cup-2014-brazil-matches-2014-to-2014-stats.csv')
wc_2018 = pd.read_csv('GitHub Data/international-fifa-world-cup-2018-russia-matches-2018-to-2018-stats.csv')
#Kian

#Sean
#friendly_2015 = pd.read_csv('GitHub Data/international-international-friendlies-matches-2015-to-2015-stats.csv')
#friendly_2016 = pd.read_csv('GitHub Data/international-international-friendlies-matches-2016-to-2016-stats.csv')
#friendly_2017 = pd.read_csv('GitHub Data/international-international-friendlies-matches-2017-to-2017-stats.csv')
friendly_2018 = pd.read_csv('GitHub Data/international-international-friendlies-matches-2018-to-2018-stats.csv')
friendly_2019 = pd.read_csv('GitHub Data/international-international-friendlies-matches-2019-to-2019-stats.csv')
friendly_2020 = pd.read_csv('GitHub Data/international-international-friendlies-matches-2020-to-2020-stats.csv')
friendly_2021 = pd.read_csv('GitHub Data/international-international-friendlies-matches-2021-to-2021-stats.csv')
friendly_2022 = pd.read_csv('GitHub Data/international-international-friendlies-matches-2022-to-2022-stats (1).csv')
#Sean

#euro_2016 = pd.read_csv('GitHub Data/international-uefa-euro-qualifiers-matches-2016-to-2016-stats.csv')
euro_2019_2020 = pd.read_csv('GitHub Data/international-uefa-euro-qualifiers-matches-2019-to-2020-stats.csv')
nations_2018_2019 = pd.read_csv('GitHub Data/international-uefa-nations-league-matches-2018-to-2019-stats.csv')
nations_2020_2021 = pd.read_csv('GitHub Data/international-uefa-nations-league-matches-2020-to-2021-stats.csv')
nations_2022_2023 = pd.read_csv('GitHub Data/international-uefa-nations-league-matches-2022-to-2023-stats.csv')

#Tyler
wc_qual_africa = pd.read_csv('GitHub Data/international-wc-qualification-africa-matches-2022-to-2022-stats.csv')
wc_qual_asia = pd.read_csv('GitHub Data/international-wc-qualification-asia-matches-2022-to-2022-stats.csv')
wc_qual_c = pd.read_csv('GitHub Data/international-wc-qualification-concacaf-matches-2022-to-2022-stats.csv')
wc_qual_euro = pd.read_csv('GitHub Data/international-wc-qualification-europe-matches-2022-to-2022-stats.csv')
wc_qual_inter = pd.read_csv('GitHub Data/international-wc-qualification-intercontinental-playoffs-matches-2022-to-2022-stats.csv')
wc_qual_sa = pd.read_csv('GitHub Data/international-wc-qualification-south-america-matches-2020-to-2022-stats.csv')

In [263]:
#TEAM DATA

#game data
asia_cup_team = pd.read_csv('GitHub team data/international-asian-cup-qualification-teams-2023-to-2023-stats.csv')
gold_cup_team = pd.read_csv('GitHub team data/international-concacaf-gold-cup-teams-2021-to-2021-stats.csv')
copa_team = pd.read_csv('GitHub team data/international-copa-america-teams-2021-to-2021-stats.csv')
arab_cup_team = pd.read_csv('GitHub team data/international-fifa-arab-cup-teams-2021-to-2021-stats.csv')
conf_cupteam = pd.read_csv('GitHub team data/international-fifa-confederations-cup-teams-2017-to-2017-stats.csv')
#wc_2002 = pd.read_csv('/work/GitHub Data/international-fifa-world-cup-2002-korea-japan-matches-2002-to-2002-stats.csv')
#wc_2006 = pd.read_csv('/work/GitHub Data/international-fifa-world-cup-2006-germany-matches-2006-to-2006-stats.csv')
#wc_2010 = pd.read_csv('/work/GitHub Data/international-fifa-world-cup-2010-south-africa-matches-2010-to-2010-stats.csv')
wc_2014_team = pd.read_csv('GitHub team data/international-fifa-world-cup-2014-brazil-teams-2014-to-2014-stats.csv')
wc_2018_team = pd.read_csv('GitHub team data/international-fifa-world-cup-2018-russia-teams-2018-to-2018-stats.csv')
friendly_2015_team = pd.read_csv('GitHub team data/international-international-friendlies-teams-2015-to-2015-stats.csv')
friendly_2016_team = pd.read_csv('GitHub team data/international-international-friendlies-teams-2016-to-2016-stats.csv')
friendly_2017_team = pd.read_csv('GitHub team data/international-international-friendlies-teams-2017-to-2017-stats.csv')
friendly_2018_team = pd.read_csv('GitHub team data/international-international-friendlies-teams-2018-to-2018-stats.csv')
friendly_2019_team = pd.read_csv('GitHub team data/international-international-friendlies-teams-2019-to-2019-stats.csv')
friendly_2020_team = pd.read_csv('GitHub team data/international-international-friendlies-teams-2020-to-2020-stats.csv')
friendly_2021_team = pd.read_csv('GitHub team data/international-international-friendlies-teams-2021-to-2021-stats.csv')
friendly_2022_team = pd.read_csv('GitHub team data/international-international-friendlies-teams-2022-to-2022-stats.csv')
#euro_2016_team = pd.read_csv('GitHub team data/international-uefa-euro-qualifiers-teams-2016-to-2016-stats.csv')
euro_2019_2020_team = pd.read_csv('GitHub team data/international-uefa-euro-qualifiers-teams-2019-to-2020-stats.csv')
nations_2018_2019_team = pd.read_csv('GitHub team data/international-uefa-nations-league-teams-2018-to-2019-stats.csv')
nations_2020_2021_team = pd.read_csv('GitHub team data/international-uefa-nations-league-teams-2020-to-2021-stats.csv')
nations_2022_2023_team = pd.read_csv('GitHub team data/international-uefa-nations-league-teams-2022-to-2023-stats.csv')
wc_qual_africa_team = pd.read_csv('GitHub team data/international-wc-qualification-africa-teams-2022-to-2022-stats.csv')
wc_qual_asia_team = pd.read_csv('GitHub team data/international-wc-qualification-asia-teams-2022-to-2022-stats.csv')
wc_qual_c_team = pd.read_csv('GitHub team data/international-wc-qualification-concacaf-teams-2022-to-2022-stats.csv')
wc_qual_euro_team = pd.read_csv('GitHub team data/international-wc-qualification-europe-teams-2022-to-2022-stats.csv')
wc_qual_inter_team = pd.read_csv('GitHub team data/international-wc-qualification-intercontinental-playoffs-teams-2022-to-2022-stats.csv')
wc_qual_sa_team = pd.read_csv('GitHub team data/international-wc-qualification-south-america-teams-2020-to-2022-stats.csv')

joined = pd.read_csv('DataScience_joined_data.csv')
qatar_teams = pd.read_csv('Qatar2022-teams.csv')

/var/folders/7r/3wbh_3yx46vflflwjp0xxbkm0000gn/T/ipykernel_83543/534271841.py:34: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  joined = pd.read_csv('DataScience_joined_data.csv')


In [264]:
#Kaggle Data
fifa_ranking = pd.read_csv('fifa_ranking-2022-10-06.csv') 
goalscorers = pd.read_csv('goalscorers.csv') 
international_matches = pd.read_csv('international_matches.csv')
last_team_scores = pd.read_csv('last_team_scores.csv')
players_22 = pd.read_csv('players_22.csv')
qatar_group_stage = pd.read_csv('Qatar_group_stage.csv')
results = pd.read_csv('results.csv')
shootouts = pd.read_csv('shootouts.csv')
squad_stats = pd.read_csv('squad_stats.csv')

/var/folders/7r/3wbh_3yx46vflflwjp0xxbkm0000gn/T/ipykernel_83543/1720374977.py:6: DtypeWarning: Columns (25,108) have mixed types. Specify dtype option on import or set low_memory=False.
  players_22 = pd.read_csv('players_22.csv')


In [265]:
# fifa_ranking['year'] = fifa_ranking['rank_date'].apply(year)
# fifa_ranking['year'] = fifa_ranking['year'].apply(int)
# fifa_ranking[fifa_ranking['year']>=2014].groupby(by='country_full').mean()
fifa_ranking = fifa_ranking.replace(['IR Iran'], 'Iran')
fifa_ranking = fifa_ranking.replace(['USA'], 'USMNT')
fifa_ranking = fifa_ranking.replace(['United Arab Emirates'], 'UAE')
fifa_ranking = fifa_ranking.replace(['Chinese Taipei'], 'Taiwan')
fifa_ranking = fifa_ranking.replace(['Kyrgyz Republic'], 'Kyrgyzstan')
fifa_ranking = fifa_ranking.replace(['Morroco A'], 'Morroco')
fifa_ranking = fifa_ranking.replace(['Türkiye'], 'Turkey')
fifa_ranking = fifa_ranking.replace(['North Macedonia'], 'FYR Macedonia')
fifa_ranking = fifa_ranking.replace(['Korea Republic'], 'South Korea')
fifa_ranking = fifa_ranking.replace(['China PR'], 'China') 
fifa_ranking = fifa_ranking.replace(['Korea DPR'], 'North Korea')


fifa_ranking['year'] = fifa_ranking['rank_date'].astype(str).apply(year)
fifa_ranking['year'] = fifa_ranking['year'].astype(float).dropna().astype(int)
fifa_ranking_2022 = fifa_ranking[fifa_ranking['rank_date']=='2022-10-06']
fifa_ranking_2022 = fifa_ranking_2022.sort_values(by='rank', ascending=True).drop(columns=['rank_date', 'confederation'])
fifa_ranking_2022

,rank,country_full,country_abrv,total_points,previous_points,rank_change,year
63809,1,Brazil,BRA,1841.30,1837.56,0,2022
63838,2,Belgium,BEL,1816.71,1821.92,0,2022
63839,3,Argentina,ARG,1773.88,1770.65,0,2022
63840,4,France,FRA,1759.78,1764.85,0,2022
63841,5,England,ENG,1728.47,1737.46,0,2022
...,...,...,...,...,...,...,...
63776,207,Sri Lanka,SRI,825.25,825.25,0,2022
63777,208,US Virgin Islands,VIR,823.97,823.97,0,2022
63778,209,British Virgin Islands,VGB,809.32,809.32,0,2022
63837,210,Anguilla,AIA,790.74,790.74,0,2022


In [266]:
fifa_ranking_2021 = fifa_ranking[fifa_ranking['rank_date']=='2021-12-23']
fifa_ranking_2021 = fifa_ranking_2021.sort_values(by='rank', ascending=True).drop(columns=['rank_date', 'confederation'])

fifa_ranking_2020 = fifa_ranking[fifa_ranking['rank_date']=='2020-12-10']
fifa_ranking_2020 = fifa_ranking_2020.sort_values(by='rank', ascending=True).drop(columns=['rank_date', 'confederation'])

fifa_ranking_2019 = fifa_ranking[fifa_ranking['rank_date']=='2019-12-19']
fifa_ranking_2019 = fifa_ranking_2019.sort_values(by='rank', ascending=True).drop(columns=['rank_date', 'confederation'])

fifa_ranking_2018 = fifa_ranking[fifa_ranking['rank_date']=='2018-12-20']
fifa_ranking_2018 = fifa_ranking_2018.sort_values(by='rank', ascending=True).drop(columns=['rank_date', 'confederation'])

In [267]:
fifa_rank_2018_to_2022 = pd.concat([fifa_ranking_2018, fifa_ranking_2019, 
fifa_ranking_2020, fifa_ranking_2021, fifa_ranking_2022])
fifa_rank_2018_to_2022

,rank,country_full,country_abrv,total_points,previous_points,rank_change,year
57608,1,Belgium,BEL,1727.00,1727.00,0,2018
57793,2,France,FRA,1726.00,1726.00,0,2018
57713,3,Brazil,BRA,1676.00,1676.00,0,2018
57607,4,Croatia,CRO,1634.00,1634.00,0,2018
57684,5,England,ENG,1631.00,1631.00,0,2018
...,...,...,...,...,...,...,...
63776,207,Sri Lanka,SRI,825.25,825.25,0,2022
63777,208,US Virgin Islands,VIR,823.97,823.97,0,2022
63778,209,British Virgin Islands,VGB,809.32,809.32,0,2022
63837,210,Anguilla,AIA,790.74,790.74,0,2022


# Data Cleaning + EDA (EDA will be done later)

In [268]:
full_data = pd.concat([asia_cup,gold_cup,copa,arab_cup,wc_2018,
                     friendly_2018, friendly_2019, friendly_2020,
                     friendly_2021, friendly_2022, euro_2019_2020, nations_2018_2019,
                     nations_2020_2021, nations_2022_2023, wc_qual_africa, wc_qual_asia, wc_qual_c,
                     wc_qual_euro, wc_qual_inter, wc_qual_sa], axis=0).reset_index()

In [269]:
year = list(full_data['date_GMT'].str.split(' ', expand = True)[2])
full_data['year'] = year
full_data['year'] = full_data['year'].apply(int)
fifa_rank_2018_to_2022['year']= fifa_rank_2018_to_2022['year'].apply(int)

In [270]:
full_data = full_data.merge(fifa_rank_2018_to_2022, left_on=['home_team_name','year'], right_on=['country_full','year'], how='left')

In [271]:
full_data = full_data.merge(fifa_rank_2018_to_2022, left_on=['away_team_name','year'], right_on=['country_full','year'], how='left',suffixes = ('_home', '_away'))

In [272]:
### Replacing null values in 'home_team_goal_timings'
full_data['home_team_goal_timings'] = full_data['home_team_goal_timings'].replace(np.nan, -1)
full_data['away_team_goal_timings'] = full_data['away_team_goal_timings'].replace(np.nan, -1)
full_data[full_data['home_team_goal_timings'].isnull()]['home_team_goal_timings']

Series([], Name: home_team_goal_timings, dtype: object)

In [273]:
full_data['days_since'] = full_data['date_GMT'].apply(time_since)
full_data = full_data[full_data['days_since']>=0]
full_data = full_data[full_data['status']!='canceled']
full_data = full_data.drop(['Game Week','referee', 'attendance', 'stadium_name'], axis=1)


In [274]:
#Checking for null values:
for i in full_data.columns:
    print(i, full_data[i].isna().sum())

index 0
timestamp 0
date_GMT 0
status 0
home_team_name 0
away_team_name 0
Pre-Match PPG (Home) 0
Pre-Match PPG (Away) 0
home_ppg 0
away_ppg 0
home_team_goal_count 0
away_team_goal_count 0
total_goal_count 0
total_goals_at_half_time 0
home_team_goal_count_half_time 0
away_team_goal_count_half_time 0
home_team_goal_timings 0
away_team_goal_timings 0
home_team_corner_count 0
away_team_corner_count 0
home_team_yellow_cards 0
home_team_red_cards 0
away_team_yellow_cards 0
away_team_red_cards 0
home_team_first_half_cards 0
home_team_second_half_cards 0
away_team_first_half_cards 0
away_team_second_half_cards 0
home_team_shots 0
away_team_shots 0
home_team_shots_on_target 0
away_team_shots_on_target 0
home_team_shots_off_target 0
away_team_shots_off_target 0
home_team_fouls 0
away_team_fouls 0
home_team_possession 0
away_team_possession 0
Home Team Pre-Match xG 0
Away Team Pre-Match xG 0
team_a_xg 0
team_b_xg 0
average_goals_per_match_pre_match 0
btts_percentage_pre_match 0
over_15_percentage

In [282]:
#Creating target variables for group and knockout stages:
full_data['home_team_result'] = full_data.apply(lambda x: winner(x.home_team_goal_count, x.away_team_goal_count), axis=1)
full_data['home_team_result_binary'] = full_data.apply(lambda x: winner_binary(x.home_team_goal_count, x.away_team_goal_count), axis=1)

In [283]:
full_data = full_data = full_data.rename(columns= {'Pre-Match PPG (Home)' : 'Pre_Match_PPG_Home', 'Pre-Match PPG (Away)' : 'Pre_Match_PPG_Away'})

In [284]:
full_data = full_data.drop(columns=['index', 'timestamp', 'status'])

At this point, full_data is completely cleaned and ready to be used for model training. Note that full_data is our match-to-match dataset of every international match since 2015.

In [285]:
text1 = full_data.columns.values.tolist()
listToStr = ','.join([str(elem) for elem in text1])
list1 = listToStr.replace(',', ' + ')
print(list1)

date_GMT + home_team_name + away_team_name + Pre_Match_PPG_Home + Pre_Match_PPG_Away + home_ppg + away_ppg + home_team_goal_count + away_team_goal_count + total_goal_count + total_goals_at_half_time + home_team_goal_count_half_time + away_team_goal_count_half_time + home_team_goal_timings + away_team_goal_timings + home_team_corner_count + away_team_corner_count + home_team_yellow_cards + home_team_red_cards + away_team_yellow_cards + away_team_red_cards + home_team_first_half_cards + home_team_second_half_cards + away_team_first_half_cards + away_team_second_half_cards + home_team_shots + away_team_shots + home_team_shots_on_target + away_team_shots_on_target + home_team_shots_off_target + away_team_shots_off_target + home_team_fouls + away_team_fouls + home_team_possession + away_team_possession + Home Team Pre-Match xG + Away Team Pre-Match xG + team_a_xg + team_b_xg + average_goals_per_match_pre_match + btts_percentage_pre_match + over_15_percentage_pre_match + over_25_percentage_p

Due to missing values in possession columns, we will take the country's average possession and impute these averages where there are no values recorded. (Need steps we took to get final_df inputted below):

In [286]:
# From Tyler's code. Need all other team data here to be concatenated for possession calcuation:
qualification_full = pd.concat([asia_cup_team,gold_cup_team,copa_team,arab_cup_team,wc_2018_team,
                     friendly_2018_team, friendly_2019_team, friendly_2020_team,
                     friendly_2021_team, friendly_2022_team, euro_2019_2020_team, nations_2018_2019_team,
                     nations_2020_2021_team, nations_2022_2023_team, wc_qual_africa_team, wc_qual_asia_team, 
                     wc_qual_c_team, wc_qual_euro_team, wc_qual_inter_team, wc_qual_sa_team])

In [287]:
qualification_full = qualification_full.reset_index().drop(columns = ['index'])
possession = qualification_full[['country', 'average_possession', 'average_possession_home', 'average_possession_away']]
possession = possession.groupby('country').agg(np.average).reset_index()
possession

,country,average_possession,average_possession_home,average_possession_away
0,Afghanistan,40.400000,29.000000,41.400000
1,Albania,51.444444,46.555556,37.666667
2,Algeria,53.857143,48.428571,44.142857
3,Andorra,37.111111,38.000000,31.333333
4,Angola,50.000000,26.750000,37.000000
...,...,...,...,...
203,Vietnam,36.800000,30.000000,15.400000
204,Wales,49.000000,48.600000,37.400000
205,Yemen,41.750000,24.000000,37.750000
206,Zambia,46.000000,25.500000,37.166667


In [288]:
### Function to find who scored first:

### HERE 1 means : home team scored first
######## 0 means : neither home team or away team scored
######## -1 means : away team scored first


first_score_country = []
who_won = []

for index, row in full_data.iterrows():
    temp_home_team_name = row['home_team_name']
    temp_away_team_name = row['away_team_name']
    
    home_first_time = str(row['home_team_goal_timings']).split(',')[0]
    away_first_time = str(row['away_team_goal_timings']).split(',')[0]
    
    if (home_first_time > away_first_time):
        first_score_country.append(1)
    elif (home_first_time < away_first_time):
        first_score_country.append(-1)
    else:
        first_score_country.append(0)
        
    temp_home_team_goal_cnt = row['home_team_goal_count']
    temp_away_team_goal_cnt = row['away_team_goal_count']
    
    if (temp_home_team_goal_cnt > temp_away_team_goal_cnt):
        who_won.append(1)
    elif (temp_home_team_goal_cnt < temp_away_team_goal_cnt):
        who_won.append(-1)
    else:
        who_won.append(0)

In [289]:
full_data['first_score_country'] = first_score_country
full_data['who_won'] = who_won

In [290]:
full_data['home_team_result'] = full_data['home_team_result'].astype(int)
full_data['home_team_result_binary'] = full_data['home_team_result_binary'].astype(int)

In [291]:
year = list(full_data['date_GMT'].str.split(' ', expand = True)[2])
full_data['year'] = year

In [292]:
year_dummies = pd.get_dummies(full_data[['year']])

In [293]:
full_data = pd.concat([full_data, year_dummies], axis = 1).drop(columns = ['year'])

In [294]:
def converting_home_and_away(df):
    '''Input : a df
    what it does : it converts home to team1 and away to team2 on the column index'''
    
    for column in list(df.columns):
        if 'home_team' in str(column):
            df = df.rename(columns = {column : column.replace('home_team', 'team1')})
        elif 'away_team' in column:
            df = df.rename(columns = {column : column.replace('away_team', 'team2')})
        elif 'home' in column:
            df = df.rename(columns = {column : column.replace('home', 'team1')})
        elif 'away' in column:
            df = df.rename(columns = {column : column.replace('away', 'team2')})
        else:
            continue
            
    return df

In [295]:
full_data = converting_home_and_away(full_data)
full_data 

,date_GMT,team1_name,team2_name,Pre_Match_PPG_Home,Pre_Match_PPG_Away,team1_ppg,team2_ppg,team1_goal_count,team2_goal_count,total_goal_count,...,days_since,team1_result,team1_result_binary,first_score_country,who_won,year_2018,year_2019,year_2020,year_2021,year_2022
0,Oct 07 2021 - 12:00pm,Indonesia,Taiwan,0.00,0.00,2.40,0.00,2,1,3,...,433,1,1,-1,1,0,0,0,1,0
1,Oct 09 2021 - 5:30pm,Guam,Cambodia,0.00,0.00,0.00,1.40,0,1,1,...,431,0,0,-1,-1,0,0,0,1,0
2,Oct 11 2021 - 1:00pm,Taiwan,Indonesia,0.00,3.00,0.00,2.40,0,3,3,...,429,0,0,-1,-1,0,0,0,1,0
3,Oct 12 2021 - 4:00pm,Cambodia,Guam,3.00,0.00,1.40,0.00,2,1,3,...,428,1,1,1,1,0,0,0,1,0
4,Jun 08 2022 - 4:30am,Philippines,Yemen,0.00,0.00,1.33,0.33,0,0,0,...,189,0,0,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3280,Mar 29 2022 - 11:30pm,Bolivia,Brazil,0.88,2.63,0.83,2.65,0,4,4,...,260,0,0,-1,-1,0,0,0,0,1
3281,Mar 29 2022 - 11:30pm,Ecuador,Argentina,1.47,2.38,1.44,2.29,1,1,2,...,260,0,0,1,0,0,0,0,0,1
3282,Mar 29 2022 - 11:30pm,Peru,Paraguay,1.24,0.94,1.33,0.89,2,0,2,...,260,1,1,1,1,0,0,0,0,1
3283,Mar 29 2022 - 11:30pm,Chile,Uruguay,1.12,1.47,1.06,1.56,0,2,2,...,260,0,0,-1,-1,0,0,0,0,1


In [300]:
full_data = full_data.dropna()

# Feature Selection here?

## VIF:

In [296]:
from statsmodels.stats.outliers_influence import variance_inflation_factor
def VIF(df, columns):
    values = sm.add_constant(df[columns]).values
    num_columns = len(columns)+1
    vif = [variance_inflation_factor(values, i) for i in range(num_columns)]
    return pd.Series(vif[1:], index=columns)

Since, we are building a model that predicts the reuslt of a game based on game statisitics, I was able to remove some relevant columns by hands. Notice that 'home_team_goal_timings' and 'away_team_goal_timings' are removed because those were used to create 'who_won' column(our dependent variable) and 'first_score_country'.

In [365]:
vif_columns = list(full_data.columns)

In [366]:
vif_columns.remove('date_GMT')
vif_columns.remove('team1_name')
vif_columns.remove('team2_name')
vif_columns.remove('team1_goal_timings')
vif_columns.remove('team2_goal_timings')
vif_columns.remove('who_won')
vif_columns.remove('team1_result_binary')
vif_columns.remove('team1_result')
vif_columns.remove('country_abrv_team2')
vif_columns.remove('country_full_team2')
vif_columns.remove('country_abrv_team1')
vif_columns.remove('country_full_team1')
vif_columns.remove('previous_points_team2')
vif_columns.remove('previous_points_team1')
vif_columns.remove('total_points_team2')
vif_columns.remove('team_a_xg')
vif_columns.remove('total_points_team1')
vif_columns.remove('days_since')
vif_columns.remove('team1_possession')
vif_columns.remove('average_goals_per_match_pre_match')
vif_columns.remove('team_b_xg')
vif_columns.remove('odds_ft_over35')

In [369]:
vif_output = VIF(full_data, vif_columns).to_frame()
vif_output.replace([np.inf, -np.inf], np.nan, inplace=True)
vif_output.dropna(inplace=True)
vif_output.index.values.tolist()
#vif_output[vif_output[0]>5].sort_values(by=0, ascending=False)
vif_output

/Users/kiangolestaneh/opt/anaconda3/lib/python3.9/site-packages/statsmodels/regression/linear_model.py:1736: RuntimeWarning: divide by zero encountered in double_scalars
  return 1 - self.ssr/self.centered_tss
/Users/kiangolestaneh/opt/anaconda3/lib/python3.9/site-packages/statsmodels/stats/outliers_influence.py:195: RuntimeWarning: divide by zero encountered in double_scalars
  vif = 1. / (1. - r_squared_i)


,0
Pre_Match_PPG_Home,2.290205
Pre_Match_PPG_Away,2.448734
team1_ppg,2.437796
team2_ppg,2.778330
team1_corner_count,2.249693
team2_corner_count,2.219151
team1_fouls,2.968776
team2_fouls,2.883360
team2_possession,6.102458
Home Team Pre-Match xG,2.174909


In [419]:
vifs = list(vif_output.reset_index()['index'])

In [420]:
vifs.append('who_won')

In [423]:
final_cols = vifs

# Splitting the Data for Modeling:

In [424]:
from sklearn.model_selection import train_test_split
import statsmodels.formula.api as smf
#Using Tyler's filtered data for now. Can go back to using original full_data if needed.
temp_full_data = full_data[final_cols]
full_data_train, full_data_test = train_test_split(temp_full_data, test_size=0.25, random_state=44)

In [425]:
y_train = full_data_train['who_won']
X_train = full_data_train.drop(['who_won'], axis=1)

y_test = full_data_test['who_won']
X_test = full_data_test.drop(['who_won'], axis=1)

# Models:

## Logistic Regression:

In [426]:
import statsmodels.api as sm
log_reg_R = sm.MNLogit(y_train, X_train).fit()
print(log_reg_R.summary())

Optimization terminated successfully.
         Current function value: 0.555398
         Iterations 8
                          MNLogit Regression Results                          
Dep. Variable:                who_won   No. Observations:                 2180
Model:                        MNLogit   Df Residuals:                     2110
Method:                           MLE   Df Model:                           68
Date:                Wed, 14 Dec 2022   Pseudo R-squ.:                  0.4736
Time:                        20:28:00   Log-Likelihood:                -1210.8
converged:                       True   LL-Null:                       -2300.3
Covariance Type:            nonrobust   LLR p-value:                     0.000
                          who_won=0       coef    std err          z      P>|z|      [0.025      0.975]
-------------------------------------------------------------------------------------------------------
Pre_Match_PPG_Home                     -0.5419      0.121 

In [427]:
from sklearn.linear_model import LogisticRegression
clf = LogisticRegression(max_iter=50)
clf.fit(X_train, y_train)

/Users/kiangolestaneh/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression(max_iter=50)

In [428]:
from sklearn.metrics import confusion_matrix
y_pred = clf.predict(X_test)
acc_logreg = clf.score(X_test, y_test)
cm = confusion_matrix(y_test, y_pred)
print(cm)
print('Accuracy:', acc_logreg)

[[134  31  38]
 [ 47  36  90]
 [ 26  18 307]]
Accuracy: 0.656121045392022


In [429]:
y_prob_logreg = clf.predict_proba(X_test)
print(y_prob_logreg)

[[0.42962017 0.30847981 0.26190003]
 [0.44278106 0.44008152 0.11713742]
 [0.0979129  0.14186131 0.76022579]
 ...
 [0.29392862 0.54041363 0.16565775]
 [0.05304196 0.12428851 0.82266953]
 [0.06486025 0.30799694 0.6271428 ]]


## Logistic Regression on final data:

In [431]:
from sklearn.linear_model import LogisticRegression
clf2 = LogisticRegression(max_iter=50)
clf2.fit(X_train, y_train)

/Users/kiangolestaneh/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression(max_iter=50)

In [433]:
from sklearn.metrics import confusion_matrix
y_pred_2 = clf2.predict(X_test)
acc_logreg2 = clf2.score(X_test, y_test)
cm = confusion_matrix(y_test, y_pred_2)
print(cm)
print('Accuracy:', acc_logreg2)

[[134  31  38]
 [ 47  36  90]
 [ 26  18 307]]
Accuracy: 0.656121045392022


In [435]:
y_prob_logreg_2 = clf2.predict_proba(X_test)
print(y_prob_logreg_2)

[[0.42962017 0.30847981 0.26190003]
 [0.44278106 0.44008152 0.11713742]
 [0.0979129  0.14186131 0.76022579]
 ...
 [0.29392862 0.54041363 0.16565775]
 [0.05304196 0.12428851 0.82266953]
 [0.06486025 0.30799694 0.6271428 ]]


## SVM:

In [436]:
from sklearn.multiclass import OneVsRestClassifier
from sklearn.svm import SVC
from sklearn.metrics import confusion_matrix
multiclass = SVC(kernel = 'linear', C = 1).fit(X_train, y_train)
svm_predictions = multiclass.predict(X_test)
acc_svm = multiclass.score(X_test, y_test)
cm = confusion_matrix(y_test, svm_predictions)
print('Accuracy:', acc_svm)
print(cm)

Accuracy: 0.7138927097661623
[[157  35  11]
 [ 47  57  69]
 [ 11  35 305]]


## LDA:

In [437]:
lda = LinearDiscriminantAnalysis()
lda.fit(X_train, y_train)

LinearDiscriminantAnalysis()

In [438]:
y_prob_lda = lda.predict_proba(X_test)
y_prob_lda

array([[0.32290657, 0.55729733, 0.1197961 ],
       [0.7933222 , 0.20165988, 0.00501792],
       [0.00190702, 0.05919213, 0.93890085],
       ...,
       [0.02022717, 0.43845112, 0.54132171],
       [0.00123339, 0.06583746, 0.93292916],
       [0.00464922, 0.18523329, 0.81011749]])

In [441]:
lda_prediction = convert_to_result(y_prob_lda)
acc_lda = accuracy_score(y_test, lda_prediction)
"Accuracy: " + str(acc_lda)

'Accuracy: 0.7138927097661623'

In [440]:
def convert_to_result(prediction):
    '''Input : nested 3D array
        Output : array of (-1,0,1)'''
    
    result = []
    for i in prediction:
        if (i[0] > i[1] and i[0] > i[2]):
            result.append(-1)
        elif (i[1] > i[0] and i[1] > i[2]):
            result.append(0)
        elif (i[2] > i[0] and i[2] > i[1]):
            result.append(1)
        else:
            result.append('Edge Case')
            
    return result
    

## Decision Tree Classifier:

In [442]:
from sklearn.tree import DecisionTreeClassifier

dtc = DecisionTreeClassifier(criterion='entropy',
                             min_samples_leaf=5, 
                             ccp_alpha=0.001,
                             random_state = 88,
                             max_depth=3)

dtc = dtc.fit(X_train, y_train)

In [443]:

y_prob_dtc = dtc.predict_proba(X_test)
y_prob_dtc

array([[0.61478599, 0.22957198, 0.15564202],
       [0.61478599, 0.22957198, 0.15564202],
       [0.12587413, 0.13986014, 0.73426573],
       ...,
       [0.2887538 , 0.27051672, 0.44072948],
       [0.        , 0.06880734, 0.93119266],
       [0.2887538 , 0.27051672, 0.44072948]])

In [444]:
dtc_prediction = convert_to_result(y_prob_dtc)
acc_dtc = accuracy_score(y_test, dtc_prediction)
"Accuracy: " + str(acc_dtc)

'Accuracy: 0.6822558459422283'

## Naive Bayes

In [445]:
from sklearn.naive_bayes import GaussianNB
gnb = GaussianNB()
y_prob_bayes_final = gnb.fit(X_train, y_train).predict_proba(X_test)

In [446]:
bayes_final_prediction = convert_to_result(y_prob_bayes_final)
acc_bayes = accuracy_score(y_test, bayes_final_prediction)
"Accuracy: " + str(acc_bayes)

'Accuracy: 0.5859697386519945'

## Amy's Model:

In [447]:
from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier(max_features=5, min_samples_leaf=5, 
                           n_estimators = 500, random_state=88, verbose=2)

rf.fit(X_train, y_train)
rf.verbose = False

y_pred = rf.predict(X_test)


acc_rf = accuracy_score(y_test,y_pred)
"Accuracy: " + str(acc_rf)

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s


building tree 1 of 500
building tree 2 of 500
building tree 3 of 500
building tree 4 of 500
building tree 5 of 500
building tree 6 of 500
building tree 7 of 500
building tree 8 of 500
building tree 9 of 500
building tree 10 of 500
building tree 11 of 500
building tree 12 of 500
building tree 13 of 500
building tree 14 of 500
building tree 15 of 500
building tree 16 of 500
building tree 17 of 500
building tree 18 of 500
building tree 19 of 500
building tree 20 of 500
building tree 21 of 500
building tree 22 of 500
building tree 23 of 500
building tree 24 of 500
building tree 25 of 500
building tree 26 of 500
building tree 27 of 500
building tree 28 of 500
building tree 29 of 500
building tree 30 of 500
building tree 31 of 500
building tree 32 of 500
building tree 33 of 500
building tree 34 of 500
building tree 35 of 500
building tree 36 of 500
building tree 37 of 500
building tree 38 of 500
building tree 39 of 500
building tree 40 of 500
building tree 41 of 500
building tree 42 of 500
b

building tree 355 of 500
building tree 356 of 500
building tree 357 of 500
building tree 358 of 500
building tree 359 of 500
building tree 360 of 500
building tree 361 of 500
building tree 362 of 500
building tree 363 of 500
building tree 364 of 500
building tree 365 of 500
building tree 366 of 500
building tree 367 of 500
building tree 368 of 500
building tree 369 of 500
building tree 370 of 500
building tree 371 of 500
building tree 372 of 500
building tree 373 of 500
building tree 374 of 500
building tree 375 of 500
building tree 376 of 500
building tree 377 of 500
building tree 378 of 500
building tree 379 of 500
building tree 380 of 500
building tree 381 of 500
building tree 382 of 500
building tree 383 of 500
building tree 384 of 500
building tree 385 of 500
building tree 386 of 500
building tree 387 of 500
building tree 388 of 500
building tree 389 of 500
building tree 390 of 500
building tree 391 of 500
building tree 392 of 500
building tree 393 of 500
building tree 394 of 500


[Parallel(n_jobs=1)]: Done 500 out of 500 | elapsed:    1.5s finished


'Accuracy: 0.7482806052269602'

In [448]:
pd.DataFrame({'Feature' : X_train.columns, 
              'Importance score': 100*rf.feature_importances_}).round(1)

,Feature,Importance score
0,Pre_Match_PPG_Home,1.7
1,Pre_Match_PPG_Away,1.6
2,team1_ppg,15.1
3,team2_ppg,10.7
4,team1_corner_count,1.5
5,team2_corner_count,1.5
6,team1_fouls,1.2
7,team2_fouls,1.1
8,team2_possession,3.0
9,Home Team Pre-Match xG,1.4


# 

# Model Accuracy Table:

In [449]:
#Once all models are done we can compare accuracies here:
models = pd.DataFrame({
    'Model': ['Support Vector Machines', 'Logistic Regression', 
              'LDA', 'Naive Bayes','Decision Tree', 'Random Forest'],
    'Accuracy': [acc_svm, acc_logreg, 
              acc_lda, acc_bayes, acc_dtc, acc_rf]})
models.round(5).sort_values(by='Accuracy', ascending=False)

,Model,Accuracy
5,Random Forest,0.74828
0,Support Vector Machines,0.71389
2,LDA,0.71389
4,Decision Tree,0.68226
1,Logistic Regression,0.65612
3,Naive Bayes,0.58597


# Aggregating Data by Country:

In [452]:
temp_agg = full_data.groupby('team1_name').agg(np.average).reset_index()
temp_agg['team1_name'].unique()

/var/folders/7r/3wbh_3yx46vflflwjp0xxbkm0000gn/T/ipykernel_83543/3825291683.py:1: FutureWarning: ['date_GMT', 'team2_name', 'team1_goal_timings', 'team2_goal_timings', 'country_full_team1', 'country_abrv_team1', 'country_full_team2', 'country_abrv_team2'] did not aggregate successfully. If any error is raised this will raise in a future version of pandas. Drop these columns/ops to avoid this warning.
  temp_agg = full_data.groupby('team1_name').agg(np.average).reset_index()


array(['Afghanistan', 'Albania', 'Algeria', 'Andorra', 'Angola',
       'Anguilla', 'Antigua and Barbuda', 'Argentina', 'Armenia', 'Aruba',
       'Australia', 'Austria', 'Azerbaijan', 'Bahamas', 'Bahrain',
       'Bangladesh', 'Barbados', 'Belarus', 'Belgium', 'Belize', 'Benin',
       'Bermuda', 'Bhutan', 'Bolivia', 'Botswana', 'Brazil',
       'British Virgin Islands', 'Bulgaria', 'Burundi', 'Cambodia',
       'Cameroon', 'Canada', 'Cayman Islands', 'Central African Republic',
       'Chad', 'Chile', 'China', 'Colombia', 'Comoros', 'Congo',
       'Congo DR', 'Costa Rica', 'Croatia', 'Cuba', 'Curaçao', 'Cyprus',
       'Czech Republic', 'Denmark', 'Djibouti', 'Dominica',
       'Dominican Republic', 'Ecuador', 'Egypt', 'El Salvador', 'England',
       'Equatorial Guinea', 'Eritrea', 'Estonia', 'Ethiopia',
       'FYR Macedonia', 'Faroe Islands', 'Fiji', 'Finland', 'France',
       'Gabon', 'Gambia', 'Georgia', 'Germany', 'Ghana', 'Gibraltar',
       'Greece', 'Grenada', 'Guam', 'Gua

# Testing Models on Specific Countries:

In [457]:
full_data

,date_GMT,team1_name,team2_name,Pre_Match_PPG_Home,Pre_Match_PPG_Away,team1_ppg,team2_ppg,team1_goal_count,team2_goal_count,total_goal_count,...,days_since,team1_result,team1_result_binary,first_score_country,who_won,year_2018,year_2019,year_2020,year_2021,year_2022
0,Oct 07 2021 - 12:00pm,Indonesia,Taiwan,0.00,0.00,2.40,0.00,2,1,3,...,433,1,1,-1,1,0,0,0,1,0
1,Oct 09 2021 - 5:30pm,Guam,Cambodia,0.00,0.00,0.00,1.40,0,1,1,...,431,0,0,-1,-1,0,0,0,1,0
2,Oct 11 2021 - 1:00pm,Taiwan,Indonesia,0.00,3.00,0.00,2.40,0,3,3,...,429,0,0,-1,-1,0,0,0,1,0
3,Oct 12 2021 - 4:00pm,Cambodia,Guam,3.00,0.00,1.40,0.00,2,1,3,...,428,1,1,1,1,0,0,0,1,0
4,Jun 08 2022 - 4:30am,Philippines,Yemen,0.00,0.00,1.33,0.33,0,0,0,...,189,0,0,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3280,Mar 29 2022 - 11:30pm,Bolivia,Brazil,0.88,2.63,0.83,2.65,0,4,4,...,260,0,0,-1,-1,0,0,0,0,1
3281,Mar 29 2022 - 11:30pm,Ecuador,Argentina,1.47,2.38,1.44,2.29,1,1,2,...,260,0,0,1,0,0,0,0,0,1
3282,Mar 29 2022 - 11:30pm,Peru,Paraguay,1.24,0.94,1.33,0.89,2,0,2,...,260,1,1,1,1,0,0,0,0,1
3283,Mar 29 2022 - 11:30pm,Chile,Uruguay,1.12,1.47,1.06,1.56,0,2,2,...,260,0,0,-1,-1,0,0,0,0,1


In [458]:
#Function that pulls all features for two specific countries that will play in a match:
def getting_data_from_team2(country_name1, country_name2):
    ### Features feeded into the model:
    ## When using X_train, values are all 0. When using X_train_final, values are positive numbers.
    ## We want to use X_train, need to figure out what is wrong:
    temp_features = X_train.columns
    temp_dict = {}
    
    for feature in list(temp_features):
        if 'team1' in feature:
            temp_dict[feature] = [list(temp_agg[temp_agg['team1_name'] == country_name1][temp_dict2.get(feature)])[0]]
        elif 'team2' in feature:
            temp_dict[feature] = [list(temp_agg[temp_agg['team1_name'] == country_name2][temp_dict2.get(feature)])[0]]
        else:
            if (feature != 'year_2022'):
                temp_dict[feature] = [0]
            else:
                temp_dict[feature] = [1]
                
    return pd.DataFrame(temp_dict)

In [461]:
vifs

['Pre_Match_PPG_Home',
 'Pre_Match_PPG_Away',
 'team1_ppg',
 'team2_ppg',
 'team1_corner_count',
 'team2_corner_count',
 'team1_fouls',
 'team2_fouls',
 'team2_possession',
 'Home Team Pre-Match xG',
 'Away Team Pre-Match xG',
 'btts_percentage_pre_match',
 'over_15_percentage_pre_match',
 'over_25_percentage_pre_match',
 'over_35_percentage_pre_match',
 'over_45_percentage_pre_match',
 'over_15_HT_FHG_percentage_pre_match',
 'over_05_HT_FHG_percentage_pre_match',
 'over_15_2HG_percentage_pre_match',
 'over_05_2HG_percentage_pre_match',
 'average_corners_per_match_pre_match',
 'average_cards_per_match_pre_match',
 'odds_ft_team1_win',
 'odds_ft_draw',
 'odds_ft_team2_win',
 'odds_ft_over15',
 'odds_ft_over25',
 'odds_ft_over45',
 'odds_btts_yes',
 'odds_btts_no',
 'rank_team1',
 'rank_change_team1',
 'rank_team2',
 'rank_change_team2',
 'first_score_country',
 'who_won']

In [ ]:
### Relevant features to be used
temp_dict2 = {'team1_ppg' : 'home_ppg', 
 'team2_ppg' : 'home_ppg', 
 'team1_goal_count' : 'home_team_goal_count', 
 'team2_goal_count': 'home_team_goal_count', 
 'team1_yellow_cards' :'home_team_yellow_cards', 
 'team1_red_cards' : 'home_team_red_cards', 
 'team2_yellow_cards' : 'home_team_yellow_cards', 
 'team2_red_cards' : 'home_team_red_cards', 
 'team1_shots': 'home_team_shots', 
 'team2_shots': 'home_team_shots',
 'team1_shots_on_target' : 'home_team_shots_on_target', 
 'team2_shots_on_target': 'home_team_shots_on_target',
 'team1_shots_off_target': 'home_team_shots_off_target', 
 'team2_shots_off_target': 'home_team_shots_off_target', 
 'team1_fouls': 'home_team_fouls',
 'team2_fouls': 'home_team_fouls', 
 'team1_possession': 'home_team_possession', 
 'team2_possession' : 'home_team_possession',
 'first_score_country' : 'first_score_country'}

In [459]:
#Testing Function between two random countries:
getting_data_from_team2('Brazil', 'South Korea')

NameError: name 'temp_dict2' is not defined

## LDA test:

In [ ]:
### Third Item indicates the prob of first country wins
### Second Item indicates the prob of draw
### first Item indicates the prob of second country wins

lda.predict_proba(getting_data_from_team2('Spain', 'Indonesia'))
#Spain has a 57% chance of winning and Indonesia has a 10% chance.

array([[0.10119773, 0.64392262, 0.25487964]])

In [ ]:
lda.predict_proba(getting_data_from_team2('England', 'France'))

array([[0.10119773, 0.64392262, 0.25487964]])

## Logistic Regression test:

The draw percentage for logistic regression seems high. Something to look at.

In [ ]:
clf.predict_proba(getting_data_from_team2('Spain', 'Indonesia'))

array([[0.32474352, 0.3237275 , 0.35152898]])

In [ ]:
clf.predict_proba(getting_data_from_team2('England', 'France'))

array([[0.32474352, 0.3237275 , 0.35152898]])

## Decision Tree Classifier test:

In [ ]:
dtc.predict_proba(getting_data_from_team2('South Korea', 'Brazil'))

array([[0.24561404, 0.43859649, 0.31578947]])

In [ ]:
dtc.predict_proba(getting_data_from_team2('Iran', 'USMNT'))

array([[0.24561404, 0.43859649, 0.31578947]])

## Naive Bayes Test

In [ ]:
gnb.predict_proba(getting_data_from_team2('Iran', 'France'))

array([[0.50998056, 0.47444906, 0.01557038]])

In [ ]:
gnb.predict_proba(getting_data_from_team2('England', 'France'))

array([[0.34837938, 0.60401188, 0.04760874]])

## Amy's Model test:

In [ ]:
rf.predict_proba(getting_data_from_team2('Spain', 'Indonesia'))


array([[0.34845154, 0.18786618, 0.46368228]])

In [ ]:
.

SyntaxError: invalid syntax (2903629900.py, line 2)

# Simulation Function:

In [ ]:
qatar_group_stage = qatar_group_stage.replace(['IR Iran'], 'Iran')
qatar_group_stage = qatar_group_stage.replace(['USA'], 'USMNT')
qatar_group_stage = qatar_group_stage.replace(['Korea Republic'], 'South Korea')
gs = qatar_group_stage.iloc[0:48, :]

In [ ]:
# Function to simulate group stage:
def group_stage_predictions2(team1, team2, model):

    team1_points = 0
    team2_points = 0

    team1_win_prob = model.predict_proba(getting_data_from_team2(team1, team2)).item(2)
    team2_win_prob = model.predict_proba(getting_data_from_team2(team1, team2)).item(0)
    draw_prob = model.predict_proba(getting_data_from_team2(team1, team2)).item(1)

    num_points = 0
    if team1_win_prob > team2_win_prob and team1_win_prob > draw_prob:
        #print(team1 + "wins with " + team1_win_prob + "probability")
        team1_points += 3
    if team2_win_prob > team1_win_prob and team2_win_prob > draw_prob:
        #print(team2 + "wins with " + team2_win_prob + "probability")
        team2_points += 3
    if draw_prob > team1_win_prob and draw_prob > team2_win_prob:
        #print('Draw')
        team2_points += 1
        team1_points += 1
    return([team1_points, team2_points])


In [ ]:
wc_group_matches = gs[['country1', 'country2', 'group']].sort_values('group')
print(wc_group_matches)

        country1      country2 group
0          Qatar       Ecuador     a
1        Senegal   Netherlands     a
35   Netherlands         Qatar     a
34       Ecuador       Senegal     a
18   Netherlands       Ecuador     a
17         Qatar       Senegal     a
2        England          Iran     b
3          USMNT         Wales     b
33          Iran         USMNT     b
32         Wales       England     b
19       England         USMNT     b
16         Wales          Iran     b
39  Saudi Arabia        Mexico     c
38        Poland     Argentina     c
21        Poland  Saudi Arabia     c
23     Argentina        Mexico     c
6         Mexico        Poland     c
7      Argentina  Saudi Arabia     c
36     Australia       Denmark     d
37       Tunisia        France     d
20       Tunisia     Australia     d
22        France       Denmark     d
4         France     Australia     d
5        Denmark       Tunisia     d
9          Spain    Costa Rica     e
42         Japan         Spain     e
2

In [ ]:
from collections import Counter

a_list = random.choices(['England', 'Draw', 'France'], weights = [0.1, 0.8, 0.1], k = 10)

counter = Counter(a_list)

most_common = counter.most_common(1)[0][0]
most_common
  # 👉️ one


'Draw'

In [ ]:
from itertools import combinations
import random

groups = qatar_group_stage['group'].unique()

# Function to simulate group stage:
def group_stage_predictions(team1, team2, model):

    team1_points = 0
    team2_points = 0

    team1_win_prob = model.predict_proba(getting_data_from_team2(team1, team2)).item(2)
    team2_win_prob = model.predict_proba(getting_data_from_team2(team1, team2)).item(0)
    draw_prob = model.predict_proba(getting_data_from_team2(team1, team2)).item(1)

    result = random.choices([team1, 'Draw', team2], weights = [team2_win_prob, draw_prob, team1_win_prob], k = 100)
    counter = Counter(result)
    most_common = counter.most_common(1)[0][0]
    if most_common == 'Draw':
        team1_points +=1
        team2_points +=1
        print('Draw')
    if most_common == team1:
        team1_points +=3
        print(team1 + " wins!")
    if most_common == team2:
        team2_points +=3
        print(team2 + " wins!")
    return [team1_points, team2_points]


In [ ]:
wc_group_matches = wc_group_matches.reset_index()

In [ ]:
gs['Points'] = gs.apply(lambda x: group_stage_predictions(x.country1, x.country2, lda_final), axis=1)
 

Ecuador wins!
Netherlands wins!
England wins!
Wales wins!
Australia wins!
Tunisia wins!
Mexico wins!
Saudi Arabia wins!
Canada wins!
Costa Rica wins!
Japan wins!
Draw
Cameroon wins!
Draw
Ghana wins!
Serbia wins!
Wales wins!
Draw
Ecuador wins!
USMNT wins!
Draw
Saudi Arabia wins!
Denmark wins!
Mexico wins!
Costa Rica wins!
Morocco wins!
Croatia wins!
Germany wins!
Cameroon wins!
Ghana wins!
Draw
Uruguay wins!
Wales wins!
USMNT wins!
Ecuador wins!
Draw
Australia wins!
Tunisia wins!
Poland wins!
Draw
Croatia wins!
Morocco wins!
Japan wins!
Costa Rica wins!
Ghana wins!
South Korea wins!
Serbia wins!
Cameroon wins!
/tmp/ipykernel_133/3946023089.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  gs['Points'] = gs.apply(lambda x: group_stage_predictions

In [ ]:
gs['country1'].unique()

array(['Qatar', 'Senegal', 'England', 'USMNT', 'France', 'Denmark',
       'Mexico', 'Argentina', 'Belgium', 'Spain', 'Germany', 'Morocco',
       'Switzerland', 'Uruguay', 'Portugal', 'Brazil', 'Wales',
       'Netherlands', 'Tunisia', 'Poland', 'Japan', 'Croatia', 'Cameroon',
       'South Korea', 'Iran', 'Ecuador', 'Australia', 'Saudi Arabia',
       'Canada', 'Costa Rica', 'Ghana', 'Serbia'], dtype=object)

In [ ]:
group_a = gs[(gs['group']=='a')]
group_b = gs[(gs['group']=='b')]
print(group_a[(group_a['country1']=='Senegal')]['Points'])
print(group_a[(group_a['country2']=='Senegal')]['Points'])

1    [0, 3]
Name: Points, dtype: object
17    [1, 1]
34    [3, 0]
Name: Points, dtype: object


In [ ]:
teams[0]

NameError: name 'teams' is not defined

In [ ]:

r = []
for team in teams:
    for i in np.arange(4):
        country = team[i]
        c = 0
        for i in gs[(gs['country1']==country)]['Points']:
            c += i[0]
        for j in gs[(gs['country2']==country)]['Points']:
            c += j[1]
        r.append(c)

In [ ]:
[r[i:i+4] for i in range(0, len(r), 4)]        # Use xrange in py2k

[[2, 5, 1, 7],
 [0, 9, 6, 3],
 [2, 1, 6, 7],
 [3, 1, 7, 6],
 [0, 4, 5, 7],
 [6, 9, 3, 0],
 [3, 1, 7, 6],
 [3, 3, 3, 9]]

In [ ]:
groups = ['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h']
teams = []
for g in groups:
    t = list(gs[(gs['group']==g)]['country1'].unique())
    teams.append(t)

In [ ]:
teams

[['Qatar', 'Senegal', 'Netherlands', 'Ecuador'],
 ['England', 'USMNT', 'Wales', 'Iran'],
 ['Mexico', 'Argentina', 'Poland', 'Saudi Arabia'],
 ['France', 'Denmark', 'Tunisia', 'Australia'],
 ['Spain', 'Germany', 'Japan', 'Costa Rica'],
 ['Belgium', 'Morocco', 'Croatia', 'Canada'],
 ['Switzerland', 'Brazil', 'Cameroon', 'Serbia'],
 ['Uruguay', 'Portugal', 'South Korea', 'Ghana']]

In [ ]:
#emp_dict  
# Calling DataFrame constructor after zipping
# both lists, with columns specified
expected_score_df = pd.DataFrame(list(zip([item for sublist in teams for item in sublist], r)),
               columns =['Country', 'Expected Score'])
expected_score_df

,Country,Expected Score
0,Qatar,2
1,Senegal,5
2,Netherlands,1
3,Ecuador,7
4,England,0
5,USMNT,9
6,Wales,6
7,Iran,3
8,Mexico,2
9,Argentina,1


In [ ]:
groups = ['a', 'a', 'a', 'a', 'b', 'b', 'b', 'b', 'c',
       'c', 'c', 'c', 'd', 'd', 'd', 'd', 'e', 'e',
       'e', 'e', 'f', 'f', 'f', 'f', 'g', 'g', 'g',
       'g', 'h', 'h', 'h', 'h']

expected_score_df['Group'] = groups
expected_score_df = expected_score_df.sort_values('Expected Score', ascending=False)
expected_score_df

,Country,Expected Score,Group
31,Ghana,9,h
21,Morocco,9,f
5,USMNT,9,b
26,Cameroon,7,g
19,Costa Rica,7,e
14,Tunisia,7,d
11,Saudi Arabia,7,c
3,Ecuador,7,a
27,Serbia,6,g
10,Poland,6,c


# Group Stage Tables

In [ ]:
group_a = pd.DataFrame(expected_score_df).where(expected_score_df['Group'] == 'a').dropna()
group_a

,Country,Expected Score,Group
3,Ecuador,7.0,a
1,Senegal,5.0,a
0,Qatar,2.0,a
2,Netherlands,1.0,a


In [ ]:
group_b = pd.DataFrame(expected_score_df).where(expected_score_df['Group'] == 'b').dropna()
group_b

,Country,Expected Score,Group
5,USMNT,9.0,b
6,Wales,6.0,b
7,Iran,3.0,b
4,England,0.0,b


In [ ]:
group_c = pd.DataFrame(expected_score_df).where(expected_score_df['Group'] == 'c').dropna()
group_c

,Country,Expected Score,Group
11,Saudi Arabia,7.0,c
10,Poland,6.0,c
8,Mexico,2.0,c
9,Argentina,1.0,c


In [ ]:
group_d = pd.DataFrame(expected_score_df).where(expected_score_df['Group'] == 'd').dropna()
group_d

,Country,Expected Score,Group
14,Tunisia,7.0,d
15,Australia,6.0,d
12,France,3.0,d
13,Denmark,1.0,d


In [ ]:
group_e = pd.DataFrame(expected_score_df).where(expected_score_df['Group'] == 'e').dropna()
group_e

,Country,Expected Score,Group
19,Costa Rica,7.0,e
18,Japan,5.0,e
17,Germany,4.0,e
16,Spain,0.0,e


In [ ]:
group_f = pd.DataFrame(expected_score_df).where(expected_score_df['Group'] == 'f').dropna()
group_f

,Country,Expected Score,Group
21,Morocco,9.0,f
20,Belgium,6.0,f
22,Croatia,3.0,f
23,Canada,0.0,f


In [ ]:
group_g = pd.DataFrame(expected_score_df).where(expected_score_df['Group'] == 'g').dropna()
group_g

,Country,Expected Score,Group
26,Cameroon,7.0,g
27,Serbia,6.0,g
24,Switzerland,3.0,g
25,Brazil,1.0,g


In [ ]:
group_h = pd.DataFrame(expected_score_df).where(expected_score_df['Group'] == 'h').dropna()
group_h

,Country,Expected Score,Group
31,Ghana,9.0,h
28,Uruguay,3.0,h
29,Portugal,3.0,h
30,South Korea,3.0,h


In [ ]:
def simulation(group_stage_df, model):
    teams = []
    r = []
    groups = ['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h']
    group = ['a', 'a', 'a', 'a', 'b', 'b', 'b', 'b', 'c',
       'c', 'c', 'c', 'd', 'd', 'd', 'd', 'e', 'e',
       'e', 'e', 'f', 'f', 'f', 'f', 'g', 'g', 'g',
       'g', 'h', 'h', 'h', 'h']
    for g in groups:
        t = list(group_stage_df[(group_stage_df['group']==g)]['country1'].unique())
        teams.append(t)
    group_stage_df['Points'] = gs.apply(lambda x: group_stage_predictions2(x.country1, x.country2, model), axis=1)
    for team in teams:
        for i in np.arange(4):
            country = team[i]
            c = 0
            for i in group_stage_df[(group_stage_df['country1']==country)]['Points']:
                c += i[0]
            for j in group_stage_df[(group_stage_df['country2']==country)]['Points']:
                c += j[1]
            r.append(c)
    expected_score_df = pd.DataFrame(list(zip([item for sublist in teams for item in sublist], r)),columns =['Country', 'Expected Score'])
    expected_score_df['Group'] = group
    expected_score_df = expected_score_df.sort_values('Expected Score', ascending=False)
    expected_score_df
    return expected_score_df
    

In [ ]:
expected_score_df = simulation(gs, lda_final)

/tmp/ipykernel_135/4005155662.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  group_stage_df['Points'] = gs.apply(lambda x: group_stage_predictions2(x.country1, x.country2, model), axis=1)


In [ ]:
pd.DataFrame(expected_score_df).where(expected_score_df['Group'] == 'a').dropna()


,Country,Expected Score,Group
1,Senegal,7.0,a
0,Qatar,5.0,a
2,Netherlands,4.0,a
3,Ecuador,0.0,a


In [ ]:
pd.DataFrame(expected_score_df).where(expected_score_df['Group'] == 'b').dropna()

,Country,Expected Score,Group
7,Iran,9.0,b
4,England,6.0,b
5,USMNT,3.0,b
6,Wales,0.0,b


In [ ]:
pd.DataFrame(expected_score_df).where(expected_score_df['Group'] == 'c').dropna()

,Country,Expected Score,Group
9,Argentina,9.0,c
10,Poland,4.0,c
8,Mexico,2.0,c
11,Saudi Arabia,1.0,c


<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=7dd733e4-f9c8-46de-9a9b-4c58af878fe3' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>